$$\text{Trabajo Final de Comunicaciones Digitales - Parte N° 1}$$

$\text{Diseño del Codificador - Decodificador }$

$$\text{Marco Teorico}$$

$\text{Codificador: es el dispositivo que toma el mensaje y produce la palabra codigo. El mensaje consise en una secuencia } $


$\text{Si queremos comunicar una secuencia de k bits. Existen } 2^k \text{secuencias distintas y cada una deberia ser mapeada a }$
$\text{una secuencia de simbolos distinta}$ $$2^k \leq m^n$$

$\text{La base matemática para lo que llamamos Codificacion binaria directa, es cuando se agrupan k bits y se interpretan}$
$\text{como un entero entre } 0 \quad y\quad 2^{k}-1$
$\text{, el cual se mapea a un simbolo dentro de un alfabeto de tamaño m = } 2^k$

$$\text{Símbolo} = \sum_{i=0}^{SF-1} b_i \cdot 2^i$$


$\text{Decodificador: es el dispositivo responsable de inferir el mensaje origial a }$
$\text{partir de la señal recibida. En el contexto}$
$\text{del canal AWGN (Ruido Blanco Gaussiano aditivo), el decodificador implementa una estrategia de Maxima Verosimilidus}$
$\text{Para un decodificador ML para el canal AWGN en tiempo discreto elige una de las secuencias de salida} x$
$\text{que maximiza: } $
$$\langle c, y \rangle - \frac{|c|^2}{2}$$

$\text{Desarrollo}$

$$\text{Modulacion de chirp por desplazamiento de frecuencia}$$

$\text{En el paper el proceso de codificación en la modulación por desplazamiento de frecuencia mediante chirps se describe matematicamete de manera precisa y puede descomponerse en tres etapa fundamentales}$

$\text{Etapa 1 ~ Mapeo de bits a simbolo decimal.}$

$\text{El proceso comienza con un vector w(nTs) compuesto por una cantidad determinada de bits, esta cantidad se denomina Spreading Factor (SF).El spreading Factor es el numero de bits por simbolos y se calcula como }𝑆𝐹=log2(𝑀), \text{donde M es la cardinalidad de la modulación (la cantidad de simbolos distintos). Por ejemplo si SF = 5 ,} hay   2^5 = 32$ 
$\text{simbolos posibles.}$ 

$\text{Una vez obtenido el vector W(nTs), se utiliza la formula planteada en el paper para sumar los valores ponderados de los bits del}$ $\text{vector w(nTs) y así generar el valor decimal entero S(nTs). Este numero se utiliza para decidir con que frecuencia inicial el chirp la trasmición.}$

$\text{Supongamos que el ancho de banda del canal que usamos para la transmisión es } B \text{ y que transmitimos  } $
$ \text {una muestra cada T, con } T = \frac{1}{B}$



$\text{Un símbolo } s(nT_s) \text{ es enviado a la entrada del modulador cada } T_s = 2^{SF} \cdot T$

$\text{El símbolo } s(nT_s) \text{ es un número real que se forma usando un vector } w(nT_s) \text{ de dígitos binarios SF, con SF como un parámetro entero }$

$\text{ llamado Factor de Expansión, el cual normalmente toma valores en } \{7,8,9,10,11,12\}$

$\text{SF(Spreading Factor) es el número de bits que se agrupan en cada símbolo. Cada número representa un entero entre } 0 \quad y \quad 2^{SF} - 1 \text{ y representa cuanto se "esparce" la señal en el tiempo y determina la cantidad de simbolos que se pueden transmitir  }$



$\text{Codificación de bists a símbolo}$
    

$\text{Implementación en python}$

<text>La función bits_to_symbols: Agrupa un arreglo plano de bits en bloques de longitud igual al Spreading Factor (SF) y 

convierte cada bloque en un número decimal. Este paso simula lo que en el paper se describe como la generación del símbolo
 
𝑠(𝑛𝑇𝑠) a partir del vector de bits 𝑤(𝑛𝑇𝑠)

In [ ]:
import random
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits en bloques de SF y los convierte en símbolos enteros."""
    bit_array = np.array(bit_array).reshape(-1, SF)
    powers = 2 ** np.arange(SF)[::-1]
    symbols = bit_array.dot(powers)
    return symbols

<text>La funcion symbols to bits: Invierte el proceso anterior: toma una lista de símbolos y los convierte en sus 
correspondientes representaciones binarias de SF bits. Esto emula la decodificación, recuperando el vector 𝑤(𝑛𝑇𝑠) desde el número 𝑠(𝑛𝑇𝑠).

In [ ]:
def symbols_to_bits(symbols, SF):
    """Convierte los símbolos de vuelta a su representación binaria."""
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    return bits.reshape(-1)

<text>  La funcion simulate_encoder_decoder: Es el núcleo de la simulación. 

1~ Genera bits aleatorios con una distribución uniforme entre 0 y 1 usando np.random.randint.

2~ Codifica: convierte esos bits en símbolos utilizando bits_to_symbols.

3~ Simula la transmisión: en este primer modelo, la transmisión es perfecta. Es decir, los símbolos recibidos son iguales a los transmitidos (sin errores de canal, por ahora).

4~ Decodifica: recupera los bits originales desde los símbolos usando symbols_to_bits.

5~ Calcula el BER: compara los bits transmitidos vs. los recibidos para calcular la tasa de error de bit (Bit Error Rate).

6~ Imprime resultados: muestra los primeros 64 bits transmitidos y decodificados, junto con el BER final y estadísticas generales.

In [ ]:
def simulate_encoder_decoder(SF, total_bits):
    """Genera bits aleatorios, codifica, decodifica y calcula el BER."""
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"

    # Generar bits aleatorios con distribución uniforme
    tx_bits = np.random.randint(0, 2, total_bits)

    # Codificación
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Transmisión simulada perfecta (sin ruido)
    rx_symbols = tx_symbols.copy()  # En canal real, se podría agregar ruido o errores

    # Decodificación
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # Cálculo de BER
    bit_errors = np.sum(tx_bits != rx_bits)
    ber = bit_errors / total_bits

    # Resultados
    print("Bits transmitidos (primeros 64):  ", tx_bits[:64])
    print("Bits decodificados (primeros 64):", rx_bits[:64])
    print(f"\nTotal de bits transmitidos: {total_bits}")
    print(f"Errores totales: {bit_errors}")
    print(f"BER (Bit Error Rate): {ber:.6f}")

# Parámetros de simulación
SF = 7  # Spreading Factor
num_bits = SF * 1000  # Enviar 1000 símbolos

simulate_encoder_decoder(SF, num_bits)